# Overview

- Kaggle の2021年10月の playground の EDA と予測モデル作成。
- logging
- EDA
    - 基本的な統計量
    - 考え方：
        - train_x と train_y の "相関"
        - train_x と test_x の分布の差の検定
    - target の偏り
- 特徴量エンジニアリング
    - 統計量: min, max, mean, std, bin count
    - 変換： log, power, normalization
    - 次元削減：k-means, PCA
    - 集約： `df.groupby(['cat_column']).agg({'continuous_column': ['min', 'max', 'mean', 'std'])
- ハイパーパラメータ調整
    - 序盤はハイパラ調整しない。
    - LightGBM Tuner で自動ハイパラ調整。
    - 細かくやるなら自前でハイパラ調整。
- 交差検証
    - 5-fold CV による予測結果のアンサンブル。

TODO
- EDA
    - Theils'U
- LightGBM
    - 計算が終わらない >4h
- XGBoost
    - [[TPS Oct 21] EDA & Modeling 🔥|Kaggle](https://www.kaggle.com/vishwas21/tps-oct-21-eda-modeling)

# References
- [Tabular Playground Series - Oct 2021](https://www.kaggle.com/c/tabular-playground-series-oct-2021)
- [[TPS-Oct] Simple EDA | Kaggle](https://www.kaggle.com/subinium/tps-oct-simple-eda)
- [TPS Oct 2021: single LightGBM | Kaggle](https://www.kaggle.com/hiro5299834/tps-oct-2021-single-lightgbm)
- [TPS September 2021 EDA | Kaggle](https://www.kaggle.com/dwin183287/tps-september-2021-eda)
- スモールサンプル（約7000）[TPS July 2021 EDA | Kaggle](https://www.kaggle.com/dwin183287/tps-july-2021-eda)
- [Kaggleで多くの実験を回すためにやっている簡単なこと | Zenn](https://zenn.dev/fkubota/articles/2b8d46b11c178ac2fa2d)
- [kaggle logger introduction | Kaggle](https://www.kaggle.com/sishihara/kaggle-logger-introduction)
- [Noob Stacking | 0.85654 | Kaggle](https://www.kaggle.com/ankitkalauni/noob-stacking-0-85654/notebook)
    - stacking に用いる12種の特徴量がほぼ同一の分布であることを確かめている。
    - それら12列の相関がかなり高い。約0.99
    - それら12列を使って学習すると、予測値の分布も、それら12列に一致
    - KDE plot と相関係数の heatmap だけ使ったとのこと
- [How to Work with Million-row Datasets Like a Pro | towards data science](https://towardsdatascience.com/how-to-work-with-million-row-datasets-like-a-pro-76fb5c381cdd)
    - 数百万行のデータロードに datatable, メモリ削減の関数、データ操作に datatable, cuDF, Dak, Vaex など。
    - Cython, numba.jit を使え、CSV よりも feather, parquet, jay を使え、など。

# Directory

In [28]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Parameters

In [29]:
DEBUG_FLAG = False
CATBOOST_FLAG = True
VERSION = 'nb01'

SUBMISSION_PATH = '/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv'
TRAIN_PATH = '/kaggle/input/tabular-playground-series-oct-2021/train.csv'
TEST_PATH = '/kaggle/input/tabular-playground-series-oct-2021/test.csv'

N_SPLITS = 2 if DEBUG_FLAG else 5
N_ESTIMATORS = 10 if DEBUG_FLAG else 200
EARLY_STOPPING_ROUNDS = 3 if DEBUG_FLAG else 20

# Install & import

In [30]:
import datetime
import pickle
import random
import sys
import time

import datatable as dt
import lightgbm as lgb
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import scipy.stats as ss
import seaborn as sns

from catboost import CatBoostClassifier
from contextlib import contextmanager
from logging import getLogger, Formatter, FileHandler, StreamHandler, INFO, DEBUG
# from optuna.integration import lightgbm as lgb
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import confusion_matrix, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler

# settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

# Functions

In [31]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')


def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df


def create_logger(exp_version):
    log_file = (f'{exp_version}.log')

    logger_ = getLogger(exp_version)
    logger_.setLevel(DEBUG)

    # formatter
    # fmr = Formatter('[%(levelname)s] %(asctime)s >>\t%(message)s')
    fmr = Formatter(
        '%(asctime)s %(name)s %(lineno)d'
        ' [%(levelname)s][%(funcName)s] %(message)s'
    )

    # file handler
    fh = FileHandler(log_file)
    fh.setLevel(DEBUG)
    fh.setFormatter(fmr)

    # stream handler
    ch = StreamHandler()
    ch.setLevel(INFO)
    ch.setFormatter(fmr)

    logger_.addHandler(fh)
    logger_.addHandler(ch)


def get_logger(exp_version):
    return getLogger(exp_version)


def get_args_of_func(f):
    return f.__code__.co_varnames[:f.__code__.co_argcount]


def show_mem_usage():
    print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
    print(" ------------------------------------ ")
    for var_name in globals():
        if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1024**2:
            print("{}{: >25}{}{: >6} MiB{}".format('|',var_name,'|', int(sys.getsizeof(eval(var_name))/1024**2),'|'))


def read_data(train_only=True):
    train = dt.fread(TRAIN_PATH).to_pandas()
    if train_only:
        test = pd.DataFrame()
    else:
        test = dt.fread(TEST_PATH).to_pandas()
    return train, test


def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))


def _fe_kmeans(df, n_clusters=6, km=None):
    cluster_cols = [f'kmeans_{i}' for i in range(n_clusters)]
    if km is None:
        km = KMeans(n_clusters=n_clusters, 
                    n_init=50, 
                    max_iter=500, 
                    random_state=0)
        df_km = km.fit_transform(df)
    else:
        df_km = km.transform(df)
    df_km = pd.DataFrame(df_km, columns=cluster_cols, index=df.index)
    return df_km, km


def _fe_pca(df, dim=6, pca=None):
    if pca is None:
        pca = PCA()
        df_pca = pca.fit_transform(df[features])
    else:
        df_pca = pca.transform(df[features])
    pca_cols = [f'pca_{i}' for i in range(df_pca.shape[1])]
    df_pca = pd.DataFrame(df_pca, columns=pca_cols, index=df.index)
    return df_pca.iloc[:, :dim], pca


def _feature_engineering(df):
    df['mean'] = df[cont_features].mean(axis=1)
    df['std'] = df[cont_features].std(axis=1)
    df['min'] = df[cont_features].min(axis=1)
    df['max'] = df[cont_features].max(axis=1)
    df['bin_count'] = df[disc_features].sum(axis=1)
    return df


def feature_engineering(df, scaler=None, km=None, pca=None):
    df = _feature_engineering(df)
    
    if scaler is None:
        scaler = RobustScaler()
        df.loc[:, ['std', 'bin_count']] = scaler.fit_transform(df[['std', 'bin_count']])
    else:
        df.loc[:, ['std', 'bin_count']] = scaler.transform(df[['std', 'bin_count']])

    df_km, km = _fe_kmeans(df[features], km=km)
    df_pca, pca = _fe_pca(df[features], pca=pca)
    df = pd.concat([df, df_km, df_pca], axis=1)
    
    return df, scaler, km, pca

# Preparing

In [32]:
create_logger(VERSION)

# Dataset overview

In [33]:
train, test = read_data(train_only=False)

if not DEBUG_FLAG:
    train = reduce_memory_usage(train)
    test = reduce_memory_usage(test) 

if DEBUG_FLAG:
    train = train.sample(n=10000)
    if len(test) != 0:
        test = test.sample(n=10000)

    
get_logger(VERSION).info(f'train.shape: {train.shape}')
get_logger(VERSION).info(f'test.shape: {test.shape}')

In [34]:
get_logger(VERSION).info('train data')
get_logger(VERSION).info(train.head(1))

get_logger(VERSION).info('test data')
get_logger(VERSION).info(test.head(1))

get_logger(VERSION).info(f'number of missing values in train: {train.isnull().sum().sum()}')
get_logger(VERSION).info(f'number of missing values in test: {test.isnull().sum().sum()}')

target_count = train['target'].value_counts()
get_logger(VERSION).info(f'target count below:')
get_logger(VERSION).info(target_count)

In [35]:
if DEBUG_FLAG:
    plt.pie(target_count, 
        labels=target_count.index,
        colors=['#76D7C4', '#F5B7B1'],
        textprops={'fontsize': 13},
        autopct='%1.1f%%')
    plt.show()

In [36]:
if DEBUG_FLAG:
    display(train.loc[:, 'f0':'f284'].describe().T.style.bar(subset=['mean'], color='#205ff2') \
        .background_gradient(subset=['std'], cmap='Reds') \
        .background_gradient(subset=['50%'], cmap='coolwarm'))

In [37]:
features = [col for col in train.columns if col not in ['id', 'target']]

cont_features = []
disc_features = []

for col in features:
    if np.issubdtype(train[col].dtype, np.floating):
        cont_features.append(col)
    else:
        disc_features.append(col)

get_logger(VERSION).info('cont. features below:')
get_logger(VERSION).info(cont_features)
get_logger(VERSION).info('disc. features below:')
get_logger(VERSION).info(disc_features)

### Correlation

||continuous data|ordinal data|categorical data|
|-:|:-|:-|:-|
|continuous data|Pearson correlation coefficient|polyserial correlation coefficient|correlation ratio|
|ordinal data||Spearman's rank correlation coefficient, <br>Kendall rank correlation coefficient, <br>Polychoric correlation coefficient|(rank correlation ratio?)|
|categorical data|||four-fold point correlation coefficient (phi coefficient), <br>Cramér's V, <br>Uncertainty coefficient (Theil's U)|

[Ref.]
- [5.5 各種手法の相互関係 | 統計学入門](http://www.snap-tck.com/room04/c01/stat/stat05/stat0505.html)

In [38]:
if DEBUG_FLAG:
    cramers_v_scores = []
    for f in disc_features:
        cramers_v_scores.append((f, cramers_v(train['target'], train[f])))

    display(pd.DataFrame(cramers_v_scores, columns=['disc_feature', 'cramers_v']) \
        .sort_values('cramers_v', ascending=False) \
        .style.bar(subset=['cramers_v'], color='#205ff2')
    )

In [39]:
if DEBUG_FLAG:
    plt.rcParams['font.size'] = 8
    fig, axes = plt.subplots(nrows=29, ncols=10, tight_layout=True, figsize=(18, 36))

    for i, ax in enumerate(axes.ravel()):
        if i < len(features):
            if f'f{i}' in cont_features:
                sns.kdeplot(train[f'f{i}'], lw=.5, alpha=.3, shade=True, ax=ax)
                sns.kdeplot(test[f'f{i}'], lw=.5, alpha=.3, shade=True, ax=ax)
            elif f'f{i}' in disc_features:
                tr = train[[f'f{i}']]
                tr = tr.assign(data = 'Train')
                te = test[[f'f{i}']]
                te = te.assign(data = 'Test')
                sns.countplot(x='data', 
                              hue=f'f{i}', 
                              order=['Train', 'Test'],
                              alpha=.3, 
                              data=pd.concat([tr, te]), 
                              ax=ax)
        else:
            ax.set_axis_off()
        
        ax.set_ylabel(None)
        
    plt.show()

In [40]:
if DEBUG_FLAG:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 15))
    sns.heatmap(train[cont_features].corr(), 
                vmin=-1.0,
                vmax=1.0,
                annot=True, 
                fmt='1.1f', 
                cmap='coolwarm', 
                ax=ax)
    plt.show()

## 相互情報量
それぞれの特徴量の、ターゲット予測因子としての有用性を確認する。  
いくつか方法はあるが、ひとつの方法として、相互情報量を計算する。  
相互情報量は、相関係数のようにふたつの特徴量間の "相関関係" を定量化する。  
相関係数が線形な関係に注目して定量化するのに対し、相互情報量は線形でなくても相関関係を定量化できる。  
相互情報量の利点は以下のとおり：
- 簡単に使えるし、簡単に説明できる
- 効率的に計算できる
- 理論的な根拠がしっかりしている
- 過学習に耐性がある（本当か？）
- 任意の種類の関係を検出できる

2つの特徴量に対する相互情報量は、一方の特徴量の値を知ることで、他方の値の不確実性がどれだけ低減するかを示す尺度である。  
相互情報量の最小の値は0で、この場合は2つの特徴量が独立な関係にあることを意味する。  
相互情報量の上限はない。

相互情報量は、その特徴量単独で予測因子としたときの有用性を確認することができる。  
複数の特徴量の相互作用によって、それらの特徴量がターゲットの予測因子として有用であるが、それぞれ単独では有用ではないことがあるので、相互情報量に意味がある。

<br><br>
定義と例を確認する。  
ふたつの離散確率変数XとYの相互情報量は以下のように定義される：
$$
I(X, Y)=\sum_{y\in Y}\sum_{x\in X}p(x, y)\log\frac{p(x, y)}{p(x)p(y)}.
$$
ここで$p(x, y)$は$X$と$Y$の同時分布関数、$p(x)$と$p(y)$はそれぞれ$X$と$Y$の周辺分布関数である。

以下は確率変数XとYが、どちらも取りうる値が$\{0, 1\}$の場合のクロス表。

|      | Y=0  | Y=1  |
| ---- | ---- | ---- |
| X=0  |$a_{00}$|$a_{01}$|
| X=1  |$a_{10}$|$a_{11}$|

総数を$N (=a_{00}+a_{01}+a_{10}+a_{11})$とすると、$p(X=0, Y=0)=a_{00}/N$, $p(X=0)=(a_{00}+a_{01})/N$, $p(Y=0)=(a_{00}+a_{10})/N$となる。  
このときの相互情報量は
$$
\begin{align}
I(X, Y)
= & p(X=0, Y=0)\log\frac{p(X=0, Y=0)}{p(X=0)p(Y=0)} \\
+ & p(X=1, Y=0)\log\frac{p(X=1, Y=0)}{p(X=1)p(Y=0)} \\
+ & p(X=0, Y=1)\log\frac{p(X=0, Y=1)}{p(X=0)p(Y=1)} \\
+ & p(X=1, Y=1)\log\frac{p(X=1, Y=1)}{p(X=1)p(Y=1)} \\[10pt]
= & \frac{a_{00}}{N}\log\frac{a_{00}/N}{(a_{00}+a_{01})/N\cdot (a_{00}+a_{10})/N} \\
+ & \frac{a_{10}}{N}\log\frac{a_{10}/N}{(a_{10}+a_{11})/N\cdot (a_{00}+a_{10})/N} \\
+ & \frac{a_{01}}{N}\log\frac{a_{01}/N}{(a_{00}+a_{01})/N\cdot (a_{01}+a_{11})/N} \\
+ & \frac{a_{11}}{N}\log\frac{a_{11}/N}{(a_{10}+a_{11})/N\cdot (a_{01}+a_{11})/N}
\end{align}
$$

極端な例を考える。 

#### 例1

|      | Y=0  | Y=1  |
| ---- | ---- | ---- |
| X=0  |   1  |   0  |
| X=1  |   0  |   1  |

このときの相互情報量は

$$
\begin{align}
I(X, Y)
=& \frac{1}{2}\log\frac{1/2}{1/2\cdot 1/2}
+  \frac{0}{2}\log\frac{0/2}{1/2\cdot 1/2}
+  \frac{0}{2}\log\frac{0/2}{1/2\cdot 1/2}
+  \frac{1}{2}\log\frac{1/2}{1/2\cdot 1/2} \\[10pt]
=& 1
\end{align}
$$

#### 例2

|      | Y=0  | Y=1  |
| ---- | ---- | ---- |
| X=0  |   1  |   1  |
| X=1  |   1  |   1  |

このときの相互情報量は

$$
\begin{align}
I(X, Y)
=& \frac{1}{4}\log\frac{1/4}{2/4\cdot 2/4}
+  \frac{1}{4}\log\frac{1/4}{2/4\cdot 2/4}
+  \frac{1}{4}\log\frac{1/4}{2/4\cdot 2/4}
+  \frac{1}{4}\log\frac{1/4}{2/4\cdot 2/4} \\[10pt]
=& 0
\end{align}
$$

#### 例3

|      | Y=0  | Y=1  |
| ---- | ---- | ---- |
| X=0  |   1  |   0  |
| X=1  |   0  |   3  |

このときの相互情報量は

$$
\begin{align}
I(X, Y)
=& \frac{1}{4}\log\frac{1/4}{1/4\cdot 1/4}
+  \frac{0}{4}\log\frac{0/4}{3/4\cdot 1/4}
+  \frac{0}{4}\log\frac{0/4}{1/4\cdot 3/4}
+  \frac{3}{4}\log\frac{3/4}{3/4\cdot 3/4} \\[10pt]
=& \frac{1}{4}\log 4 + \frac{3}{4}\log \frac{4}{3} \\[10pt]
\approx& 0.8112
\end{align}
$$

#### 例4

|      | Y=0  | Y=1  |
| ---- | ---- | ---- |
| X=0  |   1  |   0  |
| X=1  |   0  |  99  |

このときの相互情報量は

$$
\begin{align}
I(X, Y)
=& \frac{1}{100}\log\frac{1/100}{1/100\cdot 1/100}
+  \frac{0}{100}\log\frac{0/100}{99/100\cdot 1/100}
+  \frac{0}{100}\log\frac{0/100}{1/100\cdot 99/100}
+  \frac{99}{100}\log\frac{99/100}{99/100\cdot 99/100} \\[10pt]
=& \frac{1}{100}\log 100 + \frac{99}{100}\log \frac{100}{99} \\[10pt]
\approx& 0.0808
\end{align}
$$

<br><br>
scikit-learn では、ターゲットが離散データか量的データかで、使う関数が異なる。  
離散の場合は sklearn.feature_selection.mutual_info_refression である。  
連続の場合は sklearn.feature_selection.mutual_info_classif である。

[参考記事] 
- [Mutual Information | Kaggle](https://www.kaggle.com/ryanholbrook/mutual-information)
- クロス表から計算 [相互情報量を用いた特徴選択 | 人工知能に関する断創録](https://aidiary.hatenablog.com/entry/20100619/1276950312)

In [41]:
if DEBUG_FLAG:
    mi_scores = mutual_info_classif(train[features], train['target'], discrete_features=[f in disc_features for f in features])
    mi_scores = pd.Series(mi_scores, name='MI Scores', index=features)
    mi_scores = mi_scores.sort_values(ascending=False)
    
    def plot_mi_scores(scores):
        scores = scores.sort_values(ascending=True)
        width = np.arange(len(scores))
        ticks = list(scores.index)
        plt.barh(width, scores, height=.4)
        plt.yticks(width, ticks)
        plt.ylim(-1, len(scores))
        plt.title('Mutual Information Scores')
        plt.grid()

    top = 40
    # plt.style.use('seaborn-talk')
    plt.figure(figsize=(8, 8))
    plot_mi_scores(mi_scores[:top])

In [42]:
show_mem_usage()

if not DEBUG_FLAG and 'test' in globals():
    del test

# Model build

予測モデルを構築する。
ここでは勾配ブースティングのひとつであるLightGBMを用いる。  
勾配ブースティングは、弱学習器をつくり、その誤差を吸収するように弱学習器を逐次的につくる学習フレームワークである。  
弱学習器には、決定木が用いられることが多く、Gradient Boosting Decision Tree (GBDT) という。  
GBDTのなかでは、XGBoost, LightGBM, CatBoostなどが有名である。  
それらはどう異なるのか、pros, consについて調査する。

**XGBoost**  
XGBoostは、分割する特徴量を選択する際に、事前ソートによるアルゴリズムと、ヒストグラムによるアルゴリズムの2種類を使う。  
事前ソートとは、分割の際に、任意の特徴量の値に基づいてサンプルをソートし、その特徴量で分割する際の情報利得による最適な分割値を決定する。  
これにより、すべての特徴量それぞれで分割する際の最適な分割値を得る。  
ヒストグラムベースのアルゴリズムでは、それぞれの特徴量でヒストグラムを構成し、そのビンを使って分割値を求める。  
これら2つの分割アルゴリズムは、速度の点でヒストグラムベースのアルゴリズムが優れている。

**LightGBM**  
LightGBMは、分割する特徴量を選択する際に、Gradient-based One-Side Sampling (GOSS)を用いる。  
ここで "勾配" が意味しているのは、損失関数に対する接線の傾きである。■■■接線とは？要確認  
サンプルごとに勾配を計算し、勾配が大きいサンプルはそのまま残し、勾配が小さいサンプルはランダムにサンプリングする。  

**CatBoost**  
カテゴリ特徴量の扱い方に特徴がある。  
特徴量の一意な値の数が、 "one_hot_max_size" で指定した値以下のときに one hot encoding を行う。  
特徴量の一意な値の数が、 "one_hot_max_size" で指定した値よりも大きいとき Ordered TS と呼ばれる Target Encoding の処理が行われる。

ちなみに、XGBoost は LightGBM や CatBoost とは異なり、カテゴリ特徴量を内部的に扱うことはできず、人手でエンコーディングする必要がある。

|Func.|XGBoost|CatBoost|LightGBM|
|-:|:-|:-|:-|
|control overfitting|learning rate or eta, <br>max_depth, <br> min_child_weight|learning_rate, <br>depth, <br>(no such feature like min_child_weight)|learning_rate, <br>max_depth, <br>num_leaves, <br>min_data_in_leaf|
|categoriccal values|(not available)|cat_features, <br>one_hot_max_size|categorical_feature|
|controlling speed|colsample_bytree, <br>subsample, <br>n_estimators|rsm, <br>(no such parameter to subset data), <br>iteratons|feature_fraction, <br>bagging_fraction, <br>num_iterations|

[参考]
- [勾配ブースティングの基礎と最新の動向 (MIRU2020 Tutorial)](https://www.slideshare.net/RyuichiKanoh/miru2020-tutorial-237272385)
- [CatBoost vs. Light GBM vs. XGBoost | towards data science](https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db)

In [43]:
?lgb.train

In [44]:
models = []
results = []
oof_train = np.zeros((len(train),))

stkf = StratifiedKFold(n_splits=N_SPLITS, 
                       shuffle=True, 
                       random_state=0)

params_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 12,
    #'num_leaves': 500,
    'learning_rate': 0.05,
    'feature_fraction': 0.80,
    'bagging_fraction': 0.80,
    'bagging_freq': 5,
    'max_bin': 100,
    'n_jobs':-1,
    'verbose': -1,
    #'num_threads': 1,
    'lambda_l2': 1.5,
    #'min_gain_to_split': 0,
    # 'is_unbalance': True,
    #'scale_pos_weight':0.15,
    'device': 'gpu',
    'num_gpu': 1,
    'gpu_platform_id':-1,
    'gpu_device_id':-1,
    'gpu_use_dp': False,
}  
params_lgb = {
    'objective': 'binary',
    'n_estimators': 2000,
    'random_state': 0,
    'learning_rate': 0.05,
    'early_stopping_rounds': 10,
    'subsample': 0.6,
    'subsample_freq': 1,
    'colsample_bytree': 0.4,
    'reg_alpha': 10.0,
    'reg_lambda': 1e-1,
    'min_child_weight': 256,
    'min_child_samples': 20,
    'device': 'gpu',
}


params_cat = {
    'iterations': 2000, 
    'loss_function': 'Logloss', 
    'depth': 8, 
    'task_type' : 'GPU',
    'use_best_model': True,
    'eval_metric': 'AUC',
    'early_stopping_rounds': 100,
    'learning_rate': 0.05,
    'border_count': 32,
    'l2_leaf_reg': 3,
    'verbose': 100
}

y_train = train['target']
X_train = train.drop(['id', 'target'], axis=1)

get_logger(VERSION).info(f'cross validation:')

for fold_id, (train_idx, valid_idx) in enumerate(stkf.split(X_train, y_train)):
    start = time.time()
    result = {}
    
    get_logger(VERSION).info(f'||' * 40)
    get_logger(VERSION).info(f'fold_id: {fold_id}')
    
    X_tr = X_train.iloc[train_idx, :].reset_index(drop=True)
    X_val = X_train.iloc[valid_idx, :].reset_index(drop=True)
    y_tr = y_train.iloc[train_idx].reset_index(drop=True)
    y_val = y_train.iloc[valid_idx].reset_index(drop=True)
    
    get_logger(VERSION).info(f'feature engineering:')
    
    #X_tr, scaler, km, pca = feature_engineering(X_tr)
    #X_val, scaler, km, pca = feature_engineering(X_val, scaler, km, pca)
    
    get_logger(VERSION).info(' '.join(X_tr.columns))
    
    if CATBOOST_FLAG:
        model_cat = CatBoostClassifier(**params_cat)
        model_cat.fit(X_tr,
                      y_tr,
                      eval_set=[(X_val, y_val)],
                      early_stopping_rounds=10)
        
        oof_train[valid_idx] = model_cat.predict_proba(X_val)[:,1]
        models.append(model_cat)
        
    else:
        lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=disc_features)
        lgb_eval = lgb.Dataset(X_val, y_val, categorical_feature=disc_features)

        model_lgb = lgb.train(params_lgb,
                              lgb_train,
                              valid_sets=[lgb_train, lgb_eval],
                              valid_names=['Train', 'Valid'],
                              verbose_eval=100,
                              num_boost_round=500,
                              evals_result=result)

        oof_train[valid_idx] = model_lgb.predict(X_val, num_iteration=model_lgb.best_iteration)
        models.append(model_lgb)
        results.append(result)
    
    score_auc = roc_auc_score(y_val, oof_train[valid_idx])
    elapsed = time.time() - start
    get_logger(VERSION).info(f'fold {fold_id} - auc: {score_auc:.6f}, elapsed time: {elapsed:.2f} [sec]\n')

In [45]:
now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
model_name = f'trained_model_lgb_{now}.pkl'
pickle.dump(models, open(model_name, 'wb'))

In [46]:
if CATBOOST_FLAG:
    fig, axes = plt.subplots(nrows=1, ncols=len(models), tight_layout=True, figsize=(6*len(models), 4))
    for i, model in enumerate(models):
        history = model.get_evals_result()
        train_metric = history['learn']['Logloss']
        axes[i].plot(train_metric, label='train metric')
        eval_metric = history['validation']['Logloss']
        axes[i].plot(eval_metric, label='eval metric')
        axes[i].set_title(f'fold {i}')
        axes[i].set_xlabel('Iterations')
        axes[i].set_ylabel('logloss')
else:
    fig, axes = plt.subplots(nrows=1, ncols=len(results), tight_layout=True, figsize=(6*len(results), 4))
    for i, res in enumerate(results):
        lgb.plot_metric(res, ax=axes[i])

plt.show()

# Predictions

In [58]:
test = dt.fread(TEST_PATH).to_pandas()
test = test.drop(['id'], axis=1)
if DEBUG_FLAG:
    test = test.sample(n=10000)
#test, _, _, _ = feature_engineering(test, scaler, km, pca)

In [59]:
models = pickle.load(open(model_name, 'rb'))

y_preds = []
for model in models:
    y_pred = model.predict(test, prediction_type='Probability')[:,1] if CATBOOST_FLAG else model.predict(test, num_iteration=model.best_iteration) 
    y_preds.append(y_pred)

y_pred = np.array(y_preds).T.sum(axis=1) / N_SPLITS
pd.DataFrame(pd.Series(y_pred.ravel()).describe()).transpose()

In [62]:
if not DEBUG_FLAG:
    submission = pd.read_csv(SUBMISSION_PATH)
    submission['target'] = y_pred
    submission.to_csv('submission.csv', index=False)
    submission.head()